In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace
)
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Dir, gp_Pln, gp_Vec
from OCC.Display.SimpleGui import init_display
from IPython.display import display
from IPython.display import display, HTML

import numpy as np
import json



DSTV_FACE_MAP = {'I':['O','U','V'], 'U':['H','U','O'], 'L':['H','U']}

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def compute_obb(shape):
    """
    Compute the Oriented Bounding Box for a shape.
    Returns a Bnd_OBB instance.
    """
    obb = Bnd_OBB()
    brepbndlib.AddOBB(shape, obb)
    return obb

In [4]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb.Center()
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb.XHSize(), obb.YHSize(), obb.ZHSize()
    dx, dy, dz = obb.XDirection(), obb.YDirection(), obb.ZDirection()

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()


In [5]:
def draw_obb_axes(obb, viewer, scale=500):
    """
    Visualize the X, Y, Z axes of the OBB in the viewer as arrows.
    """
    from OCC.Core.gp import gp_Pnt
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

    origin_xyz = obb.Center()
    origin = gp_Pnt(origin_xyz.X(), origin_xyz.Y(), origin_xyz.Z())

    axes = [
        (obb.XDirection(), "RED"),
        (obb.YDirection(), "GREEN"),
        (obb.ZDirection(), "BLUE"),
    ]

    for dir_vec, color in axes:
        end = gp_Pnt(
            origin.X() + scale * dir_vec.X(),
            origin.Y() + scale * dir_vec.Y(),
            origin.Z() + scale * dir_vec.Z()
        )
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    viewer.FitAll()


In [6]:
def align_by_obb_geometry(shape, obb, swap_xy=False):
    """
    Aligns shape to global axes using OBB geometry:
    - Z = longest axis
    - X = wider of remaining two (flange)
    - Y = narrower (web)
    - Origin = min OBB corner → (0, 0, 0)
    """
    from OCC.Core.gp import gp_Trsf, gp_Vec, gp_Pnt, gp_Ax3, gp_Dir
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
    from OCC.Core.Bnd import Bnd_OBB
    from OCC.Core.BRepBndLib import brepbndlib_AddOBB

    # Step 1: Get all OBB directions and sizes
    hx, hy, hz = obb.XHSize(), obb.YHSize(), obb.ZHSize()
    half_sizes = [hx, hy, hz]
    dirs = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]

    # Step 2: Identify longest axis = Z
    length_idx = half_sizes.index(max(half_sizes))
    rem = [i for i in range(3) if i != length_idx]

    # Step 3: Assign width/height from remaining two
    # X = flange (wider), Y = web (narrower)
    if half_sizes[rem[0]] >= half_sizes[rem[1]]:
        x_idx, y_idx = rem[0], rem[1]
    else:
        x_idx, y_idx = rem[1], rem[0]

    if swap_xy:
        x_idx, y_idx = y_idx, x_idx

    x_dir = gp_Dir(dirs[x_idx])
    y_dir = gp_Dir(dirs[y_idx])
    z_dir = gp_Dir(dirs[length_idx])

    # Ensure right-handed coordinate system
    if x_dir.Crossed(y_dir).Dot(z_dir) < 0:
        y_dir.Reverse()

    # Step 4: Rotate shape to global frame
    center = obb.Center()
    localCS = gp_Ax3(gp_Pnt(center.X(), center.Y(), center.Z()), z_dir, x_dir)
    globalCS = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

    trsf_align = gp_Trsf()
    trsf_align.SetTransformation(localCS, globalCS)

    shape_rotated = BRepBuilderAPI_Transform(shape, trsf_align, True).Shape()

    # Step 5: Translate min corner to origin
    obb_rot = Bnd_OBB()
    brepbndlib_AddOBB(shape_rotated, obb_rot)

    c = obb_rot.Center()
    hx, hy, hz = obb_rot.XHSize(), obb_rot.YHSize(), obb_rot.ZHSize()
    dx, dy, dz = obb_rot.XDirection(), obb_rot.YDirection(), obb_rot.ZDirection()

    min_corner = gp_Pnt(
        c.X() - hx * dx.X() - hy * dy.X() - hz * dz.X(),
        c.Y() - hx * dx.Y() - hy * dy.Y() - hz * dz.Y(),
        c.Z() - hx * dx.Z() - hy * dy.Z() - hz * dz.Z()
    )

    trsf_shift = gp_Trsf()
    trsf_shift.SetTranslation(gp_Vec(min_corner, gp_Pnt(0, 0, 0)))

    shape_final = BRepBuilderAPI_Transform(shape_rotated, trsf_shift, True).Shape()
    return shape_final


In [7]:
# def align_to_geometry_frame(shape, obb, swap_xy=False):
#     """
#     Aligns shape to geometry-driven axes:
#     - Z = longest axis (length)
#     - X/Y = sorted width/height
#     - Origin = min corner (0,0,0)
#     """
#     from OCC.Core.gp import gp_Trsf, gp_Vec, gp_Pnt, gp_Ax3, gp_Dir
#     from OCC.Core.Bnd import Bnd_OBB
#     from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

#     # Get OBB info
#     hx, hy, hz = obb.XHSize(), obb.YHSize(), obb.ZHSize()
#     half_sizes = [hx, hy, hz]
#     dirs = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]

#     # Determine primary (length) axis
#     length_idx = half_sizes.index(max(half_sizes))
#     rem = [i for i in range(3) if i != length_idx]
    
#     # Assign width/height from remaining two
#     w_idx, h_idx = rem if half_sizes[rem[0]] >= half_sizes[rem[1]] else rem[::-1]

#     if swap_xy:
#         w_idx, h_idx = h_idx, w_idx

#     x_vec = dirs[w_idx]
#     y_vec = dirs[h_idx]
#     z_vec = dirs[length_idx]

#     # Right-handed correction
#     if x_vec.Crossed(y_vec).Dot(z_vec) < 0:
#         y_vec.Reverse()

#     x_dir = gp_Dir(x_vec)
#     y_dir = gp_Dir(y_vec)
#     z_dir = gp_Dir(z_vec)

#     # OBB center
#     c = obb.Center()
#     localCS = gp_Ax3(gp_Pnt(c.X(), c.Y(), c.Z()), z_dir, x_dir)
#     globalCS = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

#     trsf_align = gp_Trsf()
#     trsf_align.SetTransformation(localCS, globalCS)
#     shape_rotated = BRepBuilderAPI_Transform(shape, trsf_align, True).Shape()

#     # Shift min corner to origin
#     obb_rot = Bnd_OBB()
#     brepbndlib.AddOBB(shape_rotated, obb_rot)
#     cx, cy, cz = obb_rot.Center().Coord()
#     hx, hy, hz = obb_rot.XHSize(), obb_rot.YHSize(), obb_rot.ZHSize()
#     dx, dy, dz = obb_rot.XDirection().Coord(), obb_rot.YDirection().Coord(), obb_rot.ZDirection().Coord()

#     min_corner = gp_Pnt(
#         cx - hx * dx[0] - hy * dy[0] - hz * dz[0],
#         cy - hx * dx[1] - hy * dy[1] - hz * dz[1],
#         cz - hx * dx[2] - hy * dy[2] - hz * dz[2]
#     )

#     shift_vec = gp_Vec(min_corner, gp_Pnt(0, 0, 0))
#     trsf_shift = gp_Trsf()
#     trsf_shift.SetTranslation(shift_vec)

#     return BRepBuilderAPI_Transform(shape_rotated, trsf_shift, True).Shape()


In [8]:
# def transform_to_global_origin(shape, obb, swap_xy=False):
#     """
#     Rotates shape to align its OBB with global XYZ, and moves the start corner to (0,0,0).
#     """
#     from OCC.Core.gp import gp_Trsf, gp_Vec, gp_Pnt, gp_Ax3, gp_Dir
#     from OCC.Core.Bnd import Bnd_OBB
#     from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

#     # Get OBB directions
#     x_vec = obb.XDirection()
#     y_vec = obb.YDirection()
#     z_vec = obb.ZDirection()

#     # Optional XY swap
#     if swap_xy:
#         x_vec, y_vec = y_vec, x_vec

#     # Right-handed check
#     if x_vec.Crossed(y_vec).Dot(z_vec) < 0:
#         y_vec.Reverse()

#     x_dir = gp_Dir(x_vec)
#     y_dir = gp_Dir(y_vec)
#     z_dir = gp_Dir(z_vec)

#     # Create transformation: align OBB frame to global
#     center = obb.Center()
#     localCS = gp_Ax3(gp_Pnt(center.X(), center.Y(), center.Z()), z_dir, x_dir)
#     globalCS = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

#     trsf_align = gp_Trsf()
#     trsf_align.SetTransformation(localCS, globalCS)

#     shape_rotated = BRepBuilderAPI_Transform(shape, trsf_align, True).Shape()

#     # Compute new OBB
#     obb_rotated = Bnd_OBB()
#     brepbndlib.AddOBB(shape_rotated, obb_rotated)

#     # Compute min corner after alignment
#     cx, cy, cz = obb_rotated.Center().Coord()
#     hx, hy, hz = obb_rotated.XHSize(), obb_rotated.YHSize(), obb_rotated.ZHSize()
#     dx, dy, dz = obb_rotated.XDirection().Coord(), obb_rotated.YDirection().Coord(), obb_rotated.ZDirection().Coord()

#     min_corner = gp_Pnt(
#         cx - hx * dx[0] - hy * dy[0] - hz * dz[0],
#         cy - hx * dx[1] - hy * dy[1] - hz * dz[1],
#         cz - hx * dx[2] - hy * dy[2] - hz * dz[2]
#     )

#     # Translate min_corner to origin
#     shift = gp_Vec(min_corner, gp_Pnt(0, 0, 0))
#     trsf_shift = gp_Trsf()
#     trsf_shift.SetTranslation(shift)

#     shape_transformed = BRepBuilderAPI_Transform(shape_rotated, trsf_shift, True).Shape()
#     return shape_transformed


In [9]:
def rotate_xy_90_global(shape):
    """
    Rotates the aligned shape 90° in global XY plane (Z stays up).
    """
    from OCC.Core.gp import gp_Trsf, gp_Ax1, gp_Pnt, gp_Dir
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
    from math import pi

    axis = gp_Ax1(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1))  # Global Z
    trsf = gp_Trsf()
    trsf.SetRotation(axis, pi / 2.0)

    return BRepBuilderAPI_Transform(shape, trsf, True).Shape()


In [10]:
def fingerprint_shape(shape, obb):
    """
    Compute width, height, and CSA of the cross-section.
    Attempts wire-based CSA first; falls back to building from edges if needed.
    """

    half_sizes = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]
    length_idx = max(range(3), key=lambda i: half_sizes[i])
    dims = [2 * h for h in half_sizes]
    width, height = sorted([dims[i] for i in range(3) if i != length_idx], reverse=True)

    center_xyz = obb.Center()
    center = gp_Pnt(center_xyz.X(), center_xyz.Y(), center_xyz.Z())
    dir_map = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]
    normal = gp_Dir(dir_map[length_idx])
    plane = gp_Pln(center, normal)

    section = BRepAlgoAPI_Section(shape, plane, True)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    compound = section.Shape()

    # Try wire-based approach first
    exp = TopExp_Explorer(compound, TopAbs_WIRE)
    if exp.More():
        wire = exp.Current()
    else:
        # Fallback: build wire from edges
        edge_exp = TopExp_Explorer(compound, TopAbs_EDGE)
        wire_builder = BRepBuilderAPI_MakeWire()
        edge_count = 0
        while edge_exp.More():
            edge = edge_exp.Current()
            wire_builder.Add(edge)
            edge_exp.Next()
            edge_count += 1

        if edge_count == 0:
            raise RuntimeError("No wire or edge found in section result")
        if not wire_builder.IsDone():
            raise RuntimeError("Failed to build wire from edges")

        wire = wire_builder.Wire()

    # Build face from wire and compute area
    face = BRepBuilderAPI_MakeFace(plane, wire, True).Face()
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    area = props.Mass()
    return width, height, area


In [11]:
def visualize(shape, obb, display, color="CYAN", clear=True):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        obb: The associated OBB object for size/scale reference.
        display: OCC display handle (e.g., from init_display()).
        color: Optional color for the shape.
        clear: If True, erase all previous shapes before displaying.
    """
    # if clear:
    #     viewer.EraseAll()

    # Display the shape
    viewer.DisplayShape(shape, color=color, update=False)

    # Draw global axes
    origin = gp_Pnt(0, 0, 0)
    scale = max(2 * obb.XHSize(), 2 * obb.YHSize(), 2 * obb.ZHSize()) * 0.5

    # Axis lines
    axes = [
        ((scale, 0, 0), "RED"),
        ((0, scale, 0), "GREEN"),
        ((0, 0, scale), "BLUE")
    ]

    for vec, axis_color in axes:
        end = gp_Pnt(*vec)
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=axis_color, update=False)

    viewer.FitAll()



In [12]:
def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to handle OBB axis flips.
    """
    with open(json_path) as f:
        lib = json.load(f)
    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": cs.get("profile_type", "Unknown")
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        swapped["Swapped_dimensions"] = False
        return original
    elif swapped:
        swapped["Swapped_dimensions"] = True
        return swapped
    else:
        return None

In [13]:
def build_classification_input(obb, width, height, area):
    """
    Constructs a classification-friendly dictionary using globally aligned OBB.
    Assumes the solid is aligned with Z as the length direction.
    """
    length = 2 * obb.ZHSize()
    return {
        "span_web": height,
        "span_flange": width,
        "area": area,
        "length": length,
    }

In [14]:
def handle_swapped_dimensions(shape_orig, obb_orig, result, json_path, viewer, debug=False):
    """
    Always visualize the final aligned shape. If Swapped_dimensions is True,
    re-align using swapped XY and re-classify. Otherwise, show original.
    """
    if result.get("Swapped_dimensions"):
        print("Swapped dimensions detected — re-aligning from original shape with swapped axes.")

        # shape_final = transform_to_global_origin(shape_orig, obb_orig, swap_xy=True)
        shape_final = align_by_obb_geometry(shape_orig, obb_orig)
        obb_final = compute_obb(shape_final)

        width, height, area = fingerprint_shape(shape_final, obb_final)
        cs = build_classification_input(obb_final, width, height, area)
        result = classify_profile(cs, json_path)

        if result.get("Swapped_dimensions"):
            print("Correcting width/height to match JSON standard orientation.")
            width, height = height, width

        print(f"Final width: {width:.3f}, height: {height:.3f}, CSA: {area:.3f}")
    else:
        print("No swap needed — using original alignment.")
        # shape_final = transform_to_global_origin(shape_orig, obb_orig, swap_xy=False)
        shape_final = align_to_geometry_frame(shape_orig, obb_orig, swap_xy=False)
        obb_final = compute_obb(shape_final)

    # Display
    if debug:
        viewer.DisplayShape(shape_orig, color="BLUE", update=False)
        viewer.DisplayShape(shape_final, color="RED", update=True)
    else:
        # viewer.EraseAll()
        visualize(shape_final, obb_final, viewer, clear=False)

    return shape_final, obb_final, result


In [15]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    
    # Load shape and compute OBB once
    shape_orig = load_step(step_path)
    obb_orig = compute_obb(shape_orig)

    # visualize(shape_orig, obb_orig, viewer)
    draw_obb_box(obb_orig, viewer, color="ORANGE")
    
    # shape_aligned = transform_to_global_origin(shape_orig, obb_orig)
    shape_aligned = align_by_obb_geometry(shape_orig, obb_orig, swap_xy=False)
    obb_aligned = compute_obb(shape_aligned)
    draw_obb_axes(obb_aligned, viewer, scale=500)
    
    visualize(shape_aligned, obb_aligned, viewer)
    draw_obb_box(obb_aligned, viewer, color="YELLOW")

    # viewer.FitAll()

    # width, height, area = fingerprint_shape(shape_aligned, obb_aligned)
    # print(f"Width: {width:.3f}, Height: {height:.3f}, CSA: {area:.3f}")

    # cs = build_classification_input(obb_aligned, width, height, area)

    # result = classify_profile(cs, json_path)
    
    # shape_aligned, obb_aligned, result = handle_swapped_dimensions(
    #     shape_orig, 
    #     obb_orig, 
    #     result, 
    #     json_path, 
    #     viewer,
    #     debug=True
    # )


Many colors for color name BLUE, using first.
Many colors for color name CYAN, using first.
Many colors for color name BLUE, using first.


C:\Users\stephen.makepeace\AppData\Local\Temp\ipykernel_33556\1833955861.py:53: DeprecationWarning: Call to deprecated function brepbndlib_AddOBB since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepbndlib.AddOBB
  brepbndlib_AddOBB(shape_rotated, obb_rot)


In [16]:
import pandas as pd

def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    df = pd.DataFrame([result])

    # Round float values
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].round(precision)

    display(df.transpose().rename(columns={0: "Value"}))


print_result_table(result)

NameError: name 'result' is not defined

In [ ]:
    visualize(shape_aligned, compute_obb(shape_aligned), viewer, color="WHITE", clear=True)
    draw_obb_axes(compute_obb(shape_aligned), viewer, scale=300)